In [1]:
"""
Persona Interview System with Hugging Face Models
도구 에이전트(GPT)와 타겟 LLM(Hugging Face 모델) 간의 대화 시스템
"""

import json
import random
from typing import Dict, List, Any
from collections import defaultdict
from pathlib import Path
import os

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForVision2Seq

from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, SystemMessage

In [2]:
# ============================================================================
# 유틸리티
# ============================================================================

def load_json(filepath: str) -> Dict:
    """JSON 파일 로드"""
    with open(filepath, "r", encoding="utf-8") as f:
        return json.load(f)


def save_json(data: Dict, filepath: str):
    """JSON 파일 저장"""
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


# ============================================================================
# 도구 함수들 (Tool Agent가 사용)
# ============================================================================

PERSONA_SCHEMA_PATH = "/Users/jisu/Desktop/2025Workshop/persona_schema.json"
BRIDGING_PATH = "/Users/jisu/Desktop/2025Workshop/bridging_relationships.json"


def load_persona_definition(dummy: str = "") -> str:
    """페르소나 정의 JSON 파일을 로드합니다."""
    try:
        data = load_json(PERSONA_SCHEMA_PATH)
        return json.dumps(data, ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Error loading persona definition: {str(e)}"


def load_bridging_relationships(dummy: str = "") -> str:
    """브리징 관계 정의 JSON 파일을 로드합니다 (언어학적 정의)."""
    try:
        data = load_json(BRIDGING_PATH)
        return json.dumps(data, ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Error loading bridging relationships: {str(e)}"


def extract_bridging_from_conversation(conversation: str) -> str:
    """
    대화에서 브리징 관계를 추출합니다.
    현재는 정의를 로드해 '가능한 관계 타입 목록'만 수집하는 더미 구현입니다.
    """
    try:
        bridging_def = load_json(BRIDGING_PATH)
        bridging_types = bridging_def.get("bridging_types", {})

        result = {
            "bridging_relationships": {
                "rules": [],  # 실제 구현에서는 추출 결과를 rules 리스트에 채웁니다.
            },
            "conversation_preview": conversation[:200] + "..." if len(conversation) > 200 else conversation,
            "available_relation_types": [],
            "note": "프로덕션에서는 LLM을 사용해 실제 브리징 관계를 rules에 채우세요.",
        }

        for category, data in bridging_types.items():
            for relation in data.get("relations", []):
                result["available_relation_types"].append(
                    {
                        "category": category,
                        "type": relation["relation_type"],
                        "korean": relation["korean"],
                        "definition": relation["definition"],
                    }
                )

        return json.dumps(result, ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Error extracting bridging: {str(e)}"


def create_bridging_extraction_prompt(utterances: List[str]) -> str:
    """브리징 관계 추출을 위한 프롬프트 생성 (언어학적 정의 기반)"""
    utterances_text = "\n".join([f"{i+1}. {utt}" for i, utt in enumerate(utterances)])
    prompt = f"""Extract **Bridging Inference** relations from the following utterances.

Utterances:
{utterances_text}

Bridging Inference Definition:
- Implicit connections between concepts that require world knowledge to understand
- Resolving references that depend on conceptual relationships rather than direct mention
- Examples require understanding semantic frames, part-whole structures, or causal chains

Relation Types and Examples:

1. Mereological (부분-전체): part-of, member-of
2. Frame-related: instrument, theme, cause-of, in (location), temporal

Output JSON:
{{"bridging_relations": [{{"anchor":"...", "anaphor":"...", "relation_type":"...", "explanation":"...", "sentence_context":"..."}}]]}}
If none, return {{"bridging_relations":[]}}.
"""
    return prompt


def create_importance_graph(bridging_data: str) -> str:
    """
    브리징 관계를 기반으로 중요도 그래프를 생성합니다.
    각 노드의 중요도는 연결된 엣지 수로 결정됩니다.
    """
    try:
        data = json.loads(bridging_data)
        relationships = data.get("bridging_relationships", {}).get("rules", [])

        node_connections = defaultdict(int)
        edge_list = []

        for rule in relationships:
            from_node = rule["from"]
            to_nodes = rule["to"]
            strength = rule.get("strength", "medium")

            for to_node in to_nodes:
                node_connections[from_node] += 1
                node_connections[to_node] += 1
                edge_list.append({"from": from_node, "to": to_node, "strength": strength})

        max_connections = max(node_connections.values()) if node_connections else 1
        importance_scores = {node: count / max_connections for node, count in node_connections.items()}

        sorted_nodes = sorted(importance_scores.items(), key=lambda x: x[1], reverse=True)

        graph_result = {
            "nodes": [{"name": node, "importance": score, "connections": node_connections[node]} for node, score in sorted_nodes],
            "edges": edge_list,
            "summary": {
                "total_nodes": len(node_connections),
                "total_edges": len(edge_list),
                "most_important": sorted_nodes[0][0] if sorted_nodes else None,
                "most_important_score": sorted_nodes[0][1] if sorted_nodes else 0,
            },
        }
        return json.dumps(graph_result, ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Error creating graph: {str(e)}"


def generate_random_persona(dummy: str = "") -> str:
    """
    랜덤 페르소나를 생성합니다. (각 카테고리에서 하나 선택)
    """
    try:
        persona_def = load_json(PERSONA_SCHEMA_PATH)
        structure = persona_def["structure"]

        persona = {}

        # Social Role
        social_categories = structure["social_role"]["categories"]
        category_key = random.choice(list(social_categories.keys()))
        persona["social_role"] = random.choice(social_categories[category_key]["examples"])

        # Personality (0/1)
        personality_traits = {}
        for trait_obj in structure["personality"]["categories"]:
            trait_name = list(trait_obj.keys())[0]
            personality_traits[trait_name] = random.choice(["0", "1"])
        persona["personality"] = personality_traits

        # Background
        background = {}
        for bg_key, bg_data in structure["background"]["categories"].items():
            background[bg_key] = random.choice(bg_data["examples"])
        persona["background"] = background

        # Interests
        interests = {}
        for int_key, int_data in structure["interests"]["categories"].items():
            interests[int_key] = random.choice(int_data["examples"])
        persona["interests"] = interests

        result = {"persona": persona, "description": "Randomly generated persona with one item per category"}
        return json.dumps(result, ensure_ascii=False, indent=2)
    except Exception as e:
        fallback = {
            "error": f"{type(e).__name__}: {str(e)}",
            "persona": {
                "social_role": "student",
                "personality": {"openness": "1", "conscientiousness": "1", "extraversion": "0", "agreeableness": "1", "neuroticism": "0"},
                "background": {"education": "unknown", "culture": "unknown"},
                "interests": {"hobby": "reading"},
            },
            "description": "Fallback persona due to loading error",
        }
        return json.dumps(fallback, ensure_ascii=False, indent=2)


# ============================================================================
# Hugging Face 모델 래퍼 (MPS/CPU/CUDA 안전)
# ============================================================================

class HuggingFaceModelWrapper:
    """Hugging Face 모델을 LangChain 스타일로 사용할 수 있게 하는 래퍼"""

    def __init__(self, model_name: str, device: str = "auto"):
        """
        Args:
            model_name: HF 모델 이름
            device: "auto" | "cuda" | "mps" | "cpu"
        """
        self.model_name = model_name
        self.device = self._resolve_device(device)

        print(f"\n{'='*80}")
        print(f"Loading model: {model_name}  on device: {self.device}")
        print(f"{'='*80}")

        trust_code = "deepseek-vl2" in model_name.lower()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=trust_code)

        prefer_half = self.device in ("cuda", "mps")
        dtype_try = torch.float16 if prefer_half else torch.float32

        def _load(dtype):
            if "deepseek-vl2" in model_name.lower():
                return AutoModelForVision2Seq.from_pretrained(model_name, trust_remote_code=True, torch_dtype=dtype)
            else:
                return AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=dtype)

        try:
            self.model = _load(dtype_try)
            if self.device != "cpu":
                self.model.to(self.device)
        except Exception as e:
            print(f"⚠️ dtype {dtype_try} failed on {self.device}: {e}\n→ Reloading with float32.")
            self.model = _load(torch.float32)
            if self.device != "cpu":
                self.model.to(self.device)

        if self.tokenizer.pad_token is None and self.tokenizer.eos_token is not None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.model.eval()
        print("Model loaded successfully!")
        print(f"{'='*80}\n")

    def _resolve_device(self, device: str) -> str:
        if device == "auto":
            if torch.cuda.is_available():
                return "cuda"
            if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
                return "mps"
            return "cpu"
        return device

    def generate(self, prompt: str, max_new_tokens: int = 256, temperature: float = 0.7) -> str:
        """텍스트 생성"""
        try:
            inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            if self.device != "cpu":
                inputs = {k: v.to(self.device) for k, v in inputs.items()}

            with torch.inference_mode():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    temperature=temperature,
                    do_sample=True,
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    use_cache=True,
                )

            generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            if prompt in generated_text:
                generated_text = generated_text.replace(prompt, "").strip()
            return generated_text
        except Exception as e:
            return f"Error generating text: {str(e)}"

    def invoke(self, messages: List) -> Any:
        """LangChain 호환 invoke 메서드"""
        prompt_parts = []
        for msg in messages:
            if hasattr(msg, "content"):
                content = msg.content
                if isinstance(msg, SystemMessage):
                    prompt_parts.append(f"System: {content}")
                elif isinstance(msg, HumanMessage):
                    prompt_parts.append(f"User: {content}")

        prompt = "\n".join(prompt_parts) + "\nAssistant:"
        response_text = self.generate(prompt)

        class Response:
            def __init__(self, content):
                self.content = content

        return Response(response_text)


# ============================================================================
# 타겟 LLM 페르소나 생성
# ============================================================================

def create_target_persona() -> Dict[str, Any]:
    """타겟 LLM에 부여할 페르소나 생성 (각 유형별로 한 항목만)"""
    persona_result = json.loads(generate_random_persona())
    return persona_result["persona"]


def format_persona_for_prompt(persona: Dict[str, Any]) -> str:
    """페르소나를 프롬프트용 텍스트로 변환"""
    parts = [f"Occupation/Role: {persona['social_role']}"]
    parts.append("\nPersonality Traits (Big Five):")
    for trait, value in persona["personality"].items():
        level = "High" if value == "1" else "Low"
        parts.append(f"  - {trait}: {level}")

    parts.append("\nBackground Information:")
    for key, value in persona["background"].items():
        parts.append(f"  - {key}: {value}")

    parts.append("\nInterests and Preferences:")
    for key, value in persona["interests"].items():
        parts.append(f"  - {key}: {value}")

    return "\n".join(parts)


# ============================================================================
# @tool 래핑 (LangChain이 자동으로 schema/실행 처리)
# ============================================================================

@tool
def load_persona_definition_tool(dummy: str = "") -> str:
    """Load the persona definition JSON file from disk and return it as a pretty JSON string."""
    return load_persona_definition(dummy)

@tool
def load_bridging_relationships_tool(dummy: str = "") -> str:
    """Load linguistic bridging relationship definitions (Mereological / Frame-related) and return as JSON string."""
    return load_bridging_relationships(dummy)

@tool
def create_bridging_extraction_prompt_tool(utterances: List[str]) -> str:
    """Create a prompt for extracting bridging inference relations from the given utterances."""
    return create_bridging_extraction_prompt(utterances)

@tool
def extract_bridging_from_conversation_tool(conversation: str) -> str:
    """Extract bridging relations from a conversation using predefined linguistic categories."""
    return extract_bridging_from_conversation(conversation)

@tool
def create_importance_graph_tool(bridging_data: str) -> str:
    """Build an importance graph (nodes/edges) from bridging relationship JSON data."""
    return create_importance_graph(bridging_data)

@tool
def generate_random_persona_tool(dummy: str = "") -> str:
    """Generate a random persona by sampling one item per category from persona schema."""
    return generate_random_persona(dummy)


# ============================================================================
# 에이전트 생성 (LangChain create_agent)
# ============================================================================

def create_tool_agent(openai_api_key: str, memory: MemorySaver):
    model = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key, temperature=0.2)

    tools = [
        load_persona_definition_tool,
        load_bridging_relationships_tool,
        create_bridging_extraction_prompt_tool,
        extract_bridging_from_conversation_tool,
        create_importance_graph_tool,
        generate_random_persona_tool,
    ]

    system_message = (
        "You are a persona-analysis agent. "
        "ALWAYS use tools to: "
        "1) load persona/bridging defs, 2) extract bridging, 3) create importance graph, 4) infer persona. "
        "Return concise outputs between tool calls."
    )

    agent = create_agent(
        model=model,
        tools=tools,
        checkpointer=memory,
        state_modifier=system_message,
    )
    return agent


# ============================================================================
# 타겟 LLM 생성
# ============================================================================

def create_target_llm(model_name: str, persona: Dict[str, Any], device: str = "auto"):
    """타겟 LLM 생성 - 특정 페르소나를 가진 대화 상대 (Hugging Face 모델)"""
    persona_text = format_persona_for_prompt(persona)
    system_message = f"""You are a person with the following persona:

{persona_text}

Engage in conversation fully immersed in this persona.
When answering questions, naturally reflect the characteristics, background, and values of this persona.
Don't list persona information directly; let it emerge naturally through conversation.
Keep your responses concise and natural (2-4 sentences)."""

    model = HuggingFaceModelWrapper(model_name, device=device)
    return model, system_message

In [3]:
# ============================================================================
# 메인 실행 함수
# ============================================================================

def run_interview_system(
    openai_api_key: str,
    target_model_name: str = "Qwen/Qwen2.5-0.5B-Instruct",
    num_questions: int = 5,
    device: str = "auto",
):
    print(f"\n{'='*80}")
    print("Persona Interview System")
    print(f"Tool Agent: OpenAI model")
    print(f"Target LLM: {target_model_name}")
    print(f"{'='*80}\n")

    # 메모리/스레드 설정
    memory = MemorySaver()
    config = {"configurable": {"thread_id": "interview-session-1"}, "recursion_limit": 50}

    # 타겟 페르소나 생성
    target_persona = create_target_persona()
    print("=" * 80)
    print("Generated Target Persona:")
    print("=" * 80)
    print(json.dumps(target_persona, ensure_ascii=False, indent=2))
    print("\n")

    # 에이전트 & 타겟 LLM
    tool_agent = create_tool_agent(openai_api_key, memory)
    target_model, target_system_msg = create_target_llm(target_model_name, target_persona, device)

    conversation_history = []

    # 초기 지시: 정의 로드 및 준비
    initial_instruction = f"""Your mission:
1. First, use tools to verify persona definition and bridging relationships.
2. Then you will interview the target LLM through {num_questions} questions, one at a time.
3. After the interview, you will analyze bridging relationships and create an importance graph.
4. Finally, infer the target persona based on the collected information."""

    print("=" * 80)
    print("Interview Started")
    print("=" * 80)

    # 초기 설정(툴 자동 실행)
    tool_agent.invoke({"messages": [HumanMessage(content=initial_instruction)]}, config=config)

    # 인터뷰 루프
    for i in range(num_questions):
        print(f"\n{'='*80}\nQuestion {i+1}/{num_questions}\n{'='*80}")

        # 질문 생성
        question_prompt = f"Now ask the target the {i+1}th question. Create a natural question that can help identify their persona."
        result = tool_agent.invoke({"messages": [HumanMessage(content=question_prompt)]}, config=config)
        agent_question = result["messages"][-1].content
        print(f"[Tool Agent Question]: {agent_question}")

        # 타겟 LLM 답변
        target_messages = [SystemMessage(content=target_system_msg), HumanMessage(content=agent_question)]
        target_response = target_model.invoke(target_messages)
        target_answer = target_response.content
        print(f"[Target Answer]: {target_answer}")

        conversation_history.append({"question": agent_question, "answer": target_answer})

        # 다음 질문을 위한 피드백
        feedback_prompt = (
            f"Target's answer: {target_answer}\n\n"
            f"Analyze this answer in one short paragraph and prepare the next question."
        )
        tool_agent.invoke({"messages": [HumanMessage(content=feedback_prompt)]}, config=config)

    # 최종 분석 단계(툴 사용 순서 지정)
    print(f"\n{'='*80}\nFinal Analysis\n{'='*80}")
    conversation_text = json.dumps(conversation_history, ensure_ascii=False)

    final_analysis_prompt = f"""
The interview is complete. Use tools in this exact order:

1) load_bridging_relationships to recall relation types.
2) extract_bridging_from_conversation with the JSON conversation below.
3) create_importance_graph using the JSON returned from step 2.
4) Based on the graph, infer the target persona succinctly (3-5 sentences).

Conversation (JSON):
{conversation_text}
"""
    final_result = tool_agent.invoke({"messages": [HumanMessage(content=final_analysis_prompt)]}, config=config)
    print("\n[Agent Final Output]\n", final_result["messages"][-1].content)

    print(f"\n{'='*80}\nInterview System Completed\n{'='*80}")

    # 결과 저장
    results = {
        "target_model": target_model_name,
        "target_persona": target_persona,
        "conversation_history": conversation_history,
    }

    output_filename = f"interview_results_{target_model_name.replace('/', '_')}.json"
    save_json(results, output_filename)
    print(f"\nResults saved to '{output_filename}'")

    return results


In [4]:
# ============================================================================
# 실행 스크립트
# ============================================================================

if __name__ == "__main__":

    def load_env_file(env_path=".env"):
        """Load environment variables from .env file"""
        env_file = Path(env_path)
        if env_file.exists():
            print(f"✅ Loading environment variables from {env_path}")
            with open(env_file, "r") as f:
                for line in f:
                    line = line.strip()
                    if line and not line.startswith("#") and "=" in line:
                        key, value = line.split("=", 1)
                        key = key.strip()
                        value = value.strip().strip('"').strip("'")
                        if not os.environ.get(key):
                            os.environ[key] = value
            print("✅ Environment variables loaded successfully\n")
        else:
            print(f"⚠️  No .env file found at {env_path}")
            print("You can create one with:\n  OPENAI_API_KEY=your-key-here\n  HF_TOKEN=your-token-here\n")

    load_env_file()

    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    HF_TOKEN = os.environ.get("HF_TOKEN")

    if not OPENAI_API_KEY:
        print("❌ Error: OPENAI_API_KEY not found!")
        print("\nPlease either:")
        print("  1. Create a .env file with: OPENAI_API_KEY=your-key")
        print("  2. Set environment variable: export OPENAI_API_KEY='your-key'")
        print("  3. Edit the code to set it directly")
        raise SystemExit(1)

    print("✅ OPENAI_API_KEY found")
    print("✅ HF_TOKEN found" if HF_TOKEN else "ℹ️  HF_TOKEN not set (needed only for gated models)\n")

    # 모델 선택
    # - "Qwen/Qwen2.5-0.5B-Instruct" (권장, 권한 불필요)
    # - "meta-llama/Llama-3.2-1B" (권한 필요, HF_TOKEN 필요)
    # - "deepseek-ai/deepseek-vl2-tiny" (Vision-Language 모델)
    TARGET_MODEL = "Qwen/Qwen2.5-0.5B-Instruct"

    # Llama 모델 선택 시 HF 로그인 (선택)
    if "llama" in TARGET_MODEL.lower():
        if HF_TOKEN:
            try:
                from huggingface_hub import login
                login(token=HF_TOKEN)
                print("✅ Logged in to Hugging Face\n")
            except Exception as e:
                print(f"⚠️  Hugging Face login failed: {e}\n")
        else:
            print("⚠️  Warning: Llama model may require HF_TOKEN!\n")

    try:
        _ = run_interview_system(
            openai_api_key=OPENAI_API_KEY,
            target_model_name=TARGET_MODEL,
            num_questions=2,
            device="auto",  # "cuda", "cpu", "mps", or "auto"
        )
    except KeyboardInterrupt:
        print("\n\n⚠️  Interview interrupted by user")
    except Exception as e:
        print(f"\n\n❌ Error: {e}")
        import traceback
        traceback.print_exc()

✅ Loading environment variables from .env
✅ Environment variables loaded successfully

✅ OPENAI_API_KEY found
✅ HF_TOKEN found

Persona Interview System
Tool Agent: OpenAI model
Target LLM: Qwen/Qwen2.5-0.5B-Instruct

Generated Target Persona:
{
  "social_role": "student",
  "personality": {
    "openness": "1",
    "conscientiousness": "1",
    "extraversion": "0",
    "agreeableness": "1",
    "neuroticism": "0"
  },
  "background": {
    "education": "unknown",
    "culture": "unknown"
  },
  "interests": {
    "hobby": "reading"
  }
}




❌ Error: create_agent() got an unexpected keyword argument 'state_modifier'


Traceback (most recent call last):
  File "/var/folders/2w/lkcqygdx72s_vktqq1wmb9p40000gn/T/ipykernel_13443/3644174785.py", line 61, in <module>
    _ = run_interview_system(
        ^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/2w/lkcqygdx72s_vktqq1wmb9p40000gn/T/ipykernel_13443/2802049392.py", line 30, in run_interview_system
    tool_agent = create_tool_agent(openai_api_key, memory)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/2w/lkcqygdx72s_vktqq1wmb9p40000gn/T/ipykernel_13443/2001530841.py", line 385, in create_tool_agent
    agent = create_agent(
            ^^^^^^^^^^^^^
TypeError: create_agent() got an unexpected keyword argument 'state_modifier'
